### 사용자 정의 모델 클래스
- 부모클래스 : nn.Module
- 필수오버라이딩 :   
    *__int__() : 모델 층 구성, 설계  
    *forward() : 순방향 학습 진행 코드 구현

In [32]:
## 모듈로딩
import torch    # 텐서 관련 모듈
import torch.nn as nn # 인공신경망 관련 모듈
import torch.nn.functional as F # 인공신경망 관련 함수들 모듈 ( 손실 함수, 활성화함수 등등 )
import torch.optim as optim # 최적화 관련 모듈 ( 가중치 , 절편 빠르게 찾아주는 알고리즘 )
from torchinfo import summary # 모델 구조 및 저보 관련 모듈
from torchmetrics.regression import * # 회귀 성능 관련
from torchmetrics.classification import * # 분류 성능 관련

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [33]:
# 랜덤 고정
torch.manual_seed(1)
# 텐서 저장 및 실행 위치설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- [기본] 신경망클래스  
    * 입력층 - 입력 피쳐 고정  
    * 출력층 - 출력 타겟 고정  
    * 은닉층 - 고정  
    

In [34]:
# 모델 설계
# 사용할 데이터셋 : if 피쳐 4개 , 타겟 1개 , 회귀
# 입력층 : 입력 4개     출력 : 20개     AF  ReLU
# 은닉층 : 입력 20개    출력 : 100개    AF  ReLU
# 출력층 : 입력 100개   출력 : 1개      AF  X, Sigmoid & SoftMax

In [35]:
class MyModel(nn.Module):
    # 이느턴스 객체 생성 시 자동 호출 메서드 (콜백 함수)
    def __init__(self):
        # 1. 부모클래스 생성
        super().__init__()

        # 2. 자식클래스의 인스턴스 속성 설정
    
        self.input_layer = nn.Linear(4, 20)     # W4 + b1 => 1p, 5 * 20 = 100개 변수
        self.hidden_layer= nn.Linear(20, 100)   # W 20 + b1 => 21*100 = 2100개 변수
        self.output_layer = nn.Linear(100, 1)   # W 100 + b1 => 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백 함수 call back func : 시스템에서 호출되는 함수)# y
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x) # y= x1w1+w2w2+x3w3+x4w4 + b
        y = F.relu(y)      #  0 <= y ---> 죽은 relu ==> leakyReLu
        y = self.hidden_layer(y) # 
        y = F.relu(y)
        return  self.output_layer(y)
          
        

In [ ]:
# 입력 피쳐 수가 동적인 모델
class MyModel2(nn.Module):
    # 이느턴스 객체 생성 시 자동 호출 메서드 (콜백 함수)
    def __init__(self, in_feature):
        # 1. 부모클래스 생성
        super().__init__()

        # 2. 자식클래스의 인스턴스 속성 설정
    
        self.input_layer = nn.Linear(in_feature, 20)     # W4 + b1 => 1p, 5 * 20 = 100개 변수
        self.hidden_layer= nn.Linear(20, 100)   # W 20 + b1 => 21*100 = 2100개 변수
        self.output_layer = nn.Linear(100, 1)   # W 100 + b1 => 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백 함수 call back func : 시스템에서 호출되는 함수)# y
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x) # y= x1w1+w2w2+x3w3+x4w4 + b
        y = F.relu(y)      #  0 <= y ---> 죽은 relu ==> leakyReLu
        y = self.hidden_layer(y) # 
        y = F.relu(y)
        return  self.output_layer(y)

In [ ]:
# 입력 피쳐수, 은칙층 퍼렙트론수가 동적인 모델
# 입력 피쳐 수가 동적인 모델
class MyModel3(nn.Module):
    # 이느턴스 객체 생성 시 자동 호출 메서드 (콜백 함수)
    def __init__(self, in_feature, in_out, h_out):
        # 1. 부모클래스 생성
        super().__init__()

        # 2. 자식클래스의 인스턴스 속성 설정
    
        self.input_layer = nn.Linear(in_feature, in_out)     # W4 + b1 => 1p, 5 * 20 = 100개 변수
        self.hidden_layer= nn.Linear(in_out, h_out)   # W 20 + b1 => 21*100 = 2100개 변수
        self.output_layer = nn.Linear(h_out, 1)   # W 100 + b1 => 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백 함수 call back func : 시스템에서 호출되는 함수)# y
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x) # y= x1w1+w2w2+x3w3+x4w4 + b
        y = F.relu(y)      #  0 <= y ---> 죽은 relu ==> leakyReLu
        y = self.hidden_layer(y) # 
        y = F.relu(y)
        return  self.output_layer(y)

In [ ]:
# 은닉층의 개수가 동적인 모델
class MyModel4(nn.Module):
    # 이느턴스 객체 생성 시 자동 호출 메서드 (콜백 함수)
    def __init__(self, in_feature, in_out, h_out):
        # 1. 부모클래스 생성
        super().__init__()

        # 2. 자식클래스의 인스턴스 속성 설정
    
        self.input_layer = nn.Linear(in_feature, in_out)     # W4 + b1 => 1p, 5 * 20 = 100개 변수
        self.hidden_layer= nn.Linear(in_out, h_out)   # W 20 + b1 => 21*100 = 2100개 변수
        self.output_layer = nn.Linear(h_out, 1)   # W 100 + b1 => 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백 함수 call back func : 시스템에서 호출되는 함수)# y
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x) # y= x1w1+w2w2+x3w3+x4w4 + b
        y = F.relu(y)      #  0 <= y ---> 죽은 relu ==> leakyReLu
        y = self.hidden_layer(y) # 
        y = F.relu(y)
        return  self.output_layer(y)

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyDynamicModel(nn.Module):
    def __init__(self, in_feature, hidden_layers, out_feature=1):
        super(MyDynamicModel, self).__init__()

        # 1. 입력층 정의
        self.input_layer = nn.Linear(in_feature, hidden_layers[0] if hidden_layers else out_feature)
        
        # 2. 은닉층을 동적으로 정의 (빈 리스트가 들어오면 은닉층을 생략)
        self.hidden_layers = nn.ModuleList()

        for i in range(1, len(hidden_layers)):
            self.hidden_layers.append(nn.Linear(hidden_layers[i - 1], hidden_layers[i]))

        # 3. 출력층 정의 (은닉층이 없으면 입력층에서 바로 출력층으로 연결)
        self.output_layer = nn.Linear(hidden_layers[-1] if hidden_layers else in_feature, out_feature)

    def forward(self, x):
        # 입력층 처리
        x = F.relu(self.input_layer(x))

        # 은닉층 처리 (은닉층이 있다면 처리)
        for layer in self.hidden_layers:
            x = F.relu(layer(x))

        # 출력층 처리
        return self.output_layer(x)

# 은닉층 없이 모델을 구성 (입력층 -> 출력층만 존재)
model = MyDynamicModel(in_feature=10, hidden_layers=[10,20,30,40,50,60])
print(model)



MyDynamicModel(
  (input_layer): Linear(in_features=10, out_features=10, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=30, bias=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): Linear(in_features=40, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=60, bias=True)
  )
  (output_layer): Linear(in_features=60, out_features=1, bias=True)
)


In [36]:
## 모델 인스턴스 생성
m1 = MyModel(4)
for m in m1.named_parameters(): print(m)

('input_layer.weight', Parameter containing:
tensor([[ 0.2576, -0.2207, -0.0969,  0.2347],
        [-0.4707,  0.2999, -0.1029,  0.2544],
        [ 0.0695, -0.0612,  0.1387,  0.0247],
        [ 0.1826, -0.1949, -0.0365, -0.0450],
        [ 0.0725, -0.0020,  0.4371,  0.1556],
        [-0.1862, -0.3020, -0.0838, -0.2157],
        [-0.1602,  0.0239,  0.2981,  0.2718],
        [-0.4888,  0.3100,  0.1397,  0.4743],
        [ 0.3300, -0.4556, -0.4754, -0.2412],
        [ 0.4391, -0.0833,  0.2140, -0.2324],
        [ 0.4906, -0.2115,  0.3750,  0.0059],
        [-0.2634,  0.2570, -0.2654,  0.1471],
        [-0.1444, -0.0548, -0.4807, -0.2384],
        [ 0.2713, -0.1215,  0.4980,  0.4008],
        [-0.0234, -0.3337,  0.3045,  0.1552],
        [-0.3232,  0.3248,  0.3036,  0.4434],
        [-0.2803, -0.0823, -0.0097,  0.0730],
        [-0.3795, -0.3548,  0.2720, -0.1172],
        [ 0.2442,  0.0285,  0.1642,  0.1099],
        [ 0.1818,  0.2479, -0.4631,  0.2517]], requires_grad=True))
('input_layer

In [37]:
# 학습 진행 ==> 모델 인스턴스명 (데이터)
# 임이의 데이터
datats = torch.FloatTensor([[1,3,5,7], [2,4,6,8]])
targetts=torch.FloatTensor([[4],[5]])

# 학습
pre_y=m1(datats)

print(pre_y)

calling forward()
tensor([[-0.1395],
        [-0.1858]], grad_fn=<AddmmBackward0>)
